# Python Intro - Final Exercise


In [1]:
import pyodbc
import pandas as pd
import numpy as np

In [2]:
### In windows, Using a ODBC DNS (predefined connection name)
import pyodbc
conn = pyodbc.connect ("DSN=College;"
                      "Server=DESKTOP-AAGNMGA\SQLEXPRESS;"
                      "Database=College;"
                      "Trusted_Connection=yes;")

In [3]:
### Get the students table
students = pd.read_sql("SELECT * FROM Students;",conn)
students.head()

,StudentId,FirstName,LastName,Gender
0,162.0,Elmira,Holston,F
1,154.0,Sierra,Van Vorst,F
2,28.0,Mycah,Bingham,F
3,54.0,Alia,Jones,F
4,31.0,Chanda,Fillmore,F


In [4]:
departments = pd.read_sql("SELECT * FROM Departments;",conn)
departments.head()

,DepartmentId,DepartmentName
0,1.0,English
1,2.0,Science
2,3.0,Arts
3,4.0,Sport


In [5]:
courses = pd.read_sql("SELECT * FROM Courses;",conn)
courses.head()

,CourseId,CourseName,DepartmentId,TeacherId
0,1.0,English Begginers,1.0,15.0
1,2.0,Advanced English,1.0,15.0
2,3.0,Proffesional English,1.0,15.0
3,4.0,Mathematics 101,2.0,2.0
4,5.0,Matematics Advanced,2.0,13.0


In [6]:
classrooms = pd.read_sql("SELECT * FROM Classrooms;",conn)
classrooms.head()

,CourseId,StudentId,degree
0,1.0,202.0,92.031070
1,1.0,12.0,81.175916
2,1.0,232.0,82.396587
3,1.0,123.0,62.420449
4,1.0,274.0,72.569900


In [7]:
teachers = pd.read_sql("SELECT * FROM Teachers;",conn)
teachers.head()

,TeacherId,FirstName,LastName,Gender
0,1.0,Kaylea,Monzingo,F
1,2.0,Amanda,Dworkin,F
2,3.0,Jordyn,Baron,F
3,4.0,Katherine,Dibari,F
4,5.0,Heather,Metcalf,F


# Questions
 
 
## Q1. Count the number of students on each department

In [8]:
Q1 = classrooms.merge(courses, on = 'CourseId').merge(departments, on='DepartmentId')
Q1.head()

,CourseId,StudentId,degree,CourseName,DepartmentId,TeacherId,DepartmentName
0,1.0,202.0,92.031070,English Begginers,1.0,15.0,English
1,1.0,12.0,81.175916,English Begginers,1.0,15.0,English
2,1.0,232.0,82.396587,English Begginers,1.0,15.0,English
3,1.0,123.0,62.420449,English Begginers,1.0,15.0,English
4,1.0,274.0,72.569900,English Begginers,1.0,15.0,English


In [9]:
Q1.groupby('DepartmentName').agg({'StudentId':pd.Series.nunique})


,StudentId
DepartmentName,
Arts,137.0
English,84.0
Science,200.0
Sport,81.0


## Q2. How many students have each course of the English department and the total number of students in the department?

In [10]:
English_Courses =Q1[(Q1.DepartmentId.isin([1]))].\
                        groupby('CourseName').\
                        agg({'StudentId':pd.Series.nunique})
English_Courses.reset_index(inplace=True)
English_Courses.columns = ['Course','StudentCnt']

In [11]:
English_Department= Q1[(Q1.DepartmentId.isin([1]))].\
                        groupby('DepartmentName').\
                        agg({'StudentId':pd.Series.nunique})
English_Department.reset_index(inplace=True)
English_Department.columns = ['Course','StudentCnt']

In [12]:
English_Total = pd.concat([English_Courses,English_Department],join='outer',ignore_index=True,sort=False) 
English_Total

,Course,StudentCnt
0,Advanced English,24.0
1,English Begginers,32.0
2,Proffesional English,38.0
3,English,84.0


## Q3. How many small (<22 students) and large (22+ students) classrooms are needed for the Science department?

In [13]:
Q3 = Q1[(Q1.CourseId.between(4,19))].\
    groupby(['DepartmentName','CourseName']).\
    agg({'StudentId':pd.Series.nunique})
Q3["ClassSize"]='Small Class'
Q3.loc[Q3["StudentId"]>22,"ClassSize"]='Big Class'
Q3.head()


StudentId    ClassSize
DepartmentName CourseName                              
Science        Algebra                12.0  Small Class
               Biology                36.0    Big Class
               Chemistry              23.0    Big Class
               Computer Science       38.0    Big Class
               Data Science           30.0    Big Class

In [14]:
Q3.groupby('ClassSize').agg({'ClassSize':'count'})

,ClassSize
ClassSize,
Big Class,10
Small Class,6


## Q4. A feminist student claims that there are more male than female in the College. Justify if the argument is correct

In [15]:
students.groupby('Gender').agg({'Gender':'count'})

,Gender
Gender,
F,165
M,115


## Q5. For which courses the percentage of male/female students is over 70%?

In [16]:
Q5 = classrooms.merge(students, on = 'StudentId', how='left').merge(courses, on='CourseId', how='left')
Q5.head()

,CourseId,StudentId,degree,FirstName,LastName,Gender,CourseName,DepartmentId,TeacherId
0,1.0,202.0,92.031070,Autumn,Major,F,English Begginers,1.0,15.0
1,1.0,12.0,81.175916,Sophiya,Aponte,F,English Begginers,1.0,15.0
2,1.0,232.0,82.396587,Victoria,Kishinevsky,F,English Begginers,1.0,15.0
3,1.0,123.0,62.420449,Kelsi,Taber,F,English Begginers,1.0,15.0
4,1.0,274.0,72.569900,Samantha,Anderson,F,English Begginers,1.0,15.0


In [21]:
over_70per = Q5.groupby(['CourseName','Gender']).\
            agg({'Gender':'count'}) / Q5.groupby(['CourseName']).agg({'Gender':'count'})
over_70per

Gender
CourseName           Gender          
Advanced English     F       0.541667
                     M       0.458333
Algebra              F       0.666667
                     M       0.333333
Biology              F       0.527778
                     M       0.472222
Chemistry            F       0.666667
                     M       0.333333
Computer Science     F       0.473684
                     M       0.526316
Data Science         F       0.400000
                     M       0.600000
English Begginers    F       0.625000
                     M       0.375000
Genetics             F       0.684211
                     M       0.315789
Geometry             F       0.642857
                     M       0.357143
Gymnastics           F       0.583333
                     M       0.416667
History of Arts      F       0.531250
                     M       0.468750
Java                 F       0.625000
                     M       0.375000
Literature           F       0.525000
                     M       0.475000
Matematics Advanced  F       0.300000
                     M       0.700000
Mathematics 101      F       0.611111
                     M       0.388889
Musicology           F       0.700000
                     M       0.300000
Neurobiology         F       0.458333
                     M       0.541667
Painting             F       0.558824
                     M       0.441176
Physics              F       0.500000
                     M       0.500000
Proffesional English F       0.500000
                     M       0.500000
Python               F       0.583333
                     M       0.416667
R                    F       0.588235
                     M       0.411765
SQL                  F       0.666667
                     M       0.333333
Sculpture            F       0.708333
                     M       0.291667
Statistics           F       0.666667
                     M       0.333333
Swimming             F       0.500000
                     M       0.500000
Tenis                F       0.722222
                     M       0.277778
Theater              F       0.500000
                     M       0.500000
Volleyball           F       0.666667
                     M       0.333333

In [18]:
over_70per[(over_70per.Gender.between(0.7001,1))].\
            groupby(['CourseName']).\
            agg({'Gender':[min]})*100


,Gender
,min
CourseName,
Sculpture,70.833333
Tenis,72.222222


## Q6. For each department, how many students passed with a grades over 80?

Possibility 1: getting the number of students with at least one degree with more than 80 per department

In [30]:
Students80 = Q1[Q1.degree.between(80,100)].\
             groupby(['DepartmentName']).\
             agg({'StudentId':pd.Series.nunique})
Students80.reset_index(inplace=True)
Students80.columns = ['DepartmentName','StudentCnt']

In [26]:
TotalStudent = Q1.groupby(['DepartmentName']).\
               agg({'StudentId':pd.Series.nunique})
TotalStudent.reset_index(inplace=True)
TotalStudent.columns = ['DepartmentName','TotalStudents']


In [27]:
Above80_pct=Q1[Q1.degree.between(80,100)].\
            groupby(['DepartmentName']).\
            agg({'StudentId':pd.Series.nunique}) /\
            Q1.groupby(['DepartmentName']).agg({'StudentId':pd.Series.nunique})*100
Above80_pct.reset_index(inplace=True)
Above80_pct.columns = ['DepartmentName','StudentCntPct']
Above80_pct

,DepartmentName,StudentCntPct
0,Arts,44.525547
1,English,47.619048
2,Science,61.000000
3,Sport,48.148148


Possibility 2: getting the number of students with mean degree higher than 80 per department

In [23]:
Above80_2_cls=Q1.groupby(['DepartmentName','StudentId']).\
        agg({'degree':'mean'}) 
Above80_2_cls.reset_index(inplace=True)
Above80_2_cls.columns = ['DepartmentName','StudentCnt','degreeMean']

In [24]:
Above80_2 = Above80_2_cls[Above80_2_cls.degreeMean.between(80,100)].\
                 groupby(['DepartmentName']).\
                 agg({'StudentCnt':pd.Series.nunique})
Above80_2.reset_index(inplace=True)


In [25]:
Above80_2_pct=Above80_2_cls[Above80_2_cls.degreeMean.between(80,100)].\
                                         groupby(['DepartmentName']).\
                                         agg({'StudentCnt':pd.Series.nunique})/\
                                         Above80_2_cls.groupby(['DepartmentName']).agg({'StudentCnt':pd.Series.nunique})*100
Above80_2_pct.reset_index(inplace=True)


In [26]:
Above80_2.merge(TotalStudent, on = 'DepartmentName').merge(Above80_2_pct, on='DepartmentName')

,DepartmentName,StudentCnt_x,TotalStudents,StudentCnt_y
0,Arts,50.0,137.0,36.496350
1,English,37.0,84.0,44.047619
2,Science,86.0,200.0,43.000000
3,Sport,36.0,81.0,44.444444


## Q7. For each department, how many students passed with a grades under 60?

Possibility 1: getting the number of students with at least one degree with less than 60 per department

In [27]:
Students60 = Q1[Q1.degree.between(0,60)].\
             groupby(['DepartmentName']).\
             agg({'StudentId':pd.Series.nunique})
Students60.reset_index(inplace=True)
Students60.columns = ['DepartmentName','StudentCnt']

In [28]:
Under60=Q1[Q1.degree.between(0,60)].\
        groupby(['DepartmentName']).\
        agg({'StudentId':pd.Series.nunique}) /Q1.groupby(['DepartmentName']).agg({'StudentId':pd.Series.nunique})*100
Under60.reset_index(inplace=True)
Under60.columns = ['DepartmentName','StudentCnt']

In [29]:
Students60.merge(TotalStudent, on = 'DepartmentName').merge(Under60, on='DepartmentName')

,DepartmentName,StudentCnt_x,TotalStudents,StudentCnt_y
0,Arts,3.0,137.0,2.189781
1,English,2.0,84.0,2.380952
2,Science,7.0,200.0,3.500000
3,Sport,1.0,81.0,1.234568


possibility 2: getting the number of students with mean degree higher than 80 per department

In [30]:
Under60_2 = Above80_2_cls[Above80_2_cls.degreeMean.between(0,60)].\
                 groupby(['DepartmentName']).\
                 agg({'StudentCnt':pd.Series.nunique})
Under60_2.reset_index(inplace=True)


In [31]:
Under60_2_pct=Above80_2_cls[Above80_2_cls.degreeMean.between(0,60)].\
                                         groupby(['DepartmentName']).\
                                         agg({'StudentCnt':pd.Series.nunique})/\
                                         Above80_2_cls.groupby(['DepartmentName']).agg({'StudentCnt':pd.Series.nunique})*100
Under60_2_pct.reset_index(inplace=True)

In [32]:
Under60_2.merge(TotalStudent, on = 'DepartmentName').merge(Under60_2_pct, on='DepartmentName')

,DepartmentName,StudentCnt_x,TotalStudents,StudentCnt_y
0,Arts,3.0,137.0,2.189781
1,English,1.0,84.0,1.190476
2,Science,2.0,200.0,1.000000
3,Sport,1.0,81.0,1.234568


## Q8. Rate the teachers by their average student's grades (in descending order).

In [31]:
Q8= classrooms.merge(courses, on = 'CourseId').merge(teachers, on='TeacherId')
Q8['TeacherName'] = Q8['FirstName'] + ' ' + Q8['LastName']


In [32]:
Q8.head()

,CourseId,StudentId,degree,CourseName,DepartmentId,TeacherId,FirstName,LastName,Gender,TeacherName
0,1.0,202.0,92.031070,English Begginers,1.0,15.0,Conor,Mckittrick,M,Conor Mckittrick
1,1.0,12.0,81.175916,English Begginers,1.0,15.0,Conor,Mckittrick,M,Conor Mckittrick
2,1.0,232.0,82.396587,English Begginers,1.0,15.0,Conor,Mckittrick,M,Conor Mckittrick
3,1.0,123.0,62.420449,English Begginers,1.0,15.0,Conor,Mckittrick,M,Conor Mckittrick
4,1.0,274.0,72.569900,English Begginers,1.0,15.0,Conor,Mckittrick,M,Conor Mckittrick


In [34]:
Q8.groupby(['TeacherName',]).\
agg({'degree':'mean'}).sort_values(by='degree', ascending=False)


,degree
TeacherName,
Darick Hess,81.432703
Jordyn Baron,80.729494
Heather Metcalf,80.440242
Zachary Healy,79.365867
Kaylea Monzingo,79.365537
Daniel Bland,78.908281
Keegan Price,78.814631
Conor Mckittrick,78.699492
Maxwell Romkee,78.673509


## Q9. Create a dataframe showing the courses, departments they are associated with, the teacher in each course, and the number of students enrolled in the course (for each course, department and teacher show the names).

In [35]:
Q9 = courses.merge(departments, on = 'DepartmentId', how='left').\
             merge(classrooms, on='CourseId', how='left').\
             merge(teachers, on='TeacherId', how='left')

In [36]:
Q9.groupby(['CourseId','CourseName','DepartmentName','FirstName','LastName']).\
   agg({'StudentId':pd.Series.nunique})

,,,,,StudentId
CourseId,CourseName,DepartmentName,FirstName,LastName,
1.0,English Begginers,English,Conor,Mckittrick,32.0
2.0,Advanced English,English,Conor,Mckittrick,24.0
3.0,Proffesional English,English,Conor,Mckittrick,38.0
4.0,Mathematics 101,Science,Amanda,Dworkin,18.0
5.0,Matematics Advanced,Science,Alexander,Watts,10.0
6.0,Algebra,Science,Alexander,Watts,12.0
7.0,Geometry,Science,Alexander,Watts,13.0
8.0,Chemistry,Science,Jeffrey,Batzel,23.0
9.0,Physics,Science,Alexander,Watts,12.0


## Q10. Create a dataframe showing the students, the number of courses they take, the average of the grades per class, and their overall average (for each student show the student name).

In [37]:
Q10 = students.merge(classrooms, on = 'StudentId', how='left').\
               merge(courses, on='CourseId', how='left')
Q10.head()

,StudentId,FirstName,LastName,Gender,CourseId,degree,CourseName,DepartmentId,TeacherId
0,162.0,Elmira,Holston,F,8.0,65.987050,Chemistry,2.0,14.0
1,162.0,Elmira,Holston,F,20.0,79.476271,History of Arts,3.0,8.0
2,162.0,Elmira,Holston,F,31.0,74.546820,Gymnastics,4.0,23.0
3,154.0,Sierra,Van Vorst,F,6.0,84.137367,Algebra,2.0,13.0
4,154.0,Sierra,Van Vorst,F,18.0,81.048019,R,2.0,17.0


In [38]:
Overall_Average = Q10.groupby(['StudentId','FirstName','LastName']).\
                      agg({'CourseId':pd.Series.nunique,'degree':'mean'})
Overall_Average.reset_index(inplace=True)
Overall_Average.columns = ['StudentId','FirstName','LastName','CourseTaken','Overall_Degree']
Overall_Average.head()

,StudentId,FirstName,LastName,CourseTaken,Overall_Degree
0,1.0,Alexandria,Heller,4.0,81.337515
1,2.0,Cassie,Liska,5.0,79.137364
2,3.0,Hayley,Frank,3.0,78.793700
3,4.0,Maxwell,Kotch,3.0,71.481003
4,5.0,Dylan,Boehm,3.0,76.718650


In [39]:
English =  Q10[(Q10.DepartmentId.isin([1]))].\
                groupby('StudentId').\
                agg({'degree':'mean'})

English.reset_index(inplace=True)
English.columns = ['StudentId','English_Degree']
English.head()


,StudentId,English_Degree
0,3.0,83.972245
1,4.0,70.929801
2,8.0,89.073972
3,12.0,83.145779
4,18.0,82.552181


In [40]:
Science =  Q10[(Q10.DepartmentId.isin([2]))].\
                groupby('StudentId').\
                agg({'degree':'mean'})

Science.reset_index(inplace=True)
Science.columns = ['StudentId','Science_Degree']
Science.head()

,StudentId,Science_Degree
0,1.0,79.557508
1,2.0,78.867115
2,3.0,76.204427
3,4.0,74.157066
4,5.0,100.000000


In [41]:
Art =  Q10[(Q10.DepartmentId.isin([3]))].\
                groupby('StudentId').\
                agg({'degree':'mean'})

Art.reset_index(inplace=True)
Art.columns = ['StudentId','Art_Degree']
Art.head()

,StudentId,Art_Degree
0,1.0,86.677535
1,2.0,79.542738
2,4.0,69.356144
3,5.0,64.494845
4,6.0,68.799129


In [42]:
Sport =  Q10[(Q10.DepartmentId.isin([4]))].\
                groupby('StudentId').\
                agg({'degree':'mean'})

Sport.reset_index(inplace=True)
Sport.columns = ['StudentId','Sport_Degree']
Sport.head()

,StudentId,Sport_Degree
0,5.0,65.661106
1,10.0,85.120393
2,13.0,80.134422
3,15.0,67.795261
4,16.0,78.507968


In [45]:
OMG_I_Did_It = Overall_Average.merge(English, on = 'StudentId', how='left').\
                               merge(Science, on = 'StudentId', how='left').\
                               merge(Art, on = 'StudentId', how='left').\
                               merge(Sport, on = 'StudentId', how='left')
OMG_I_Did_It.head()

,StudentId,FirstName,LastName,CourseTaken,Overall_Degree,English_Degree,Science_Degree,Art_Degree,Sport_Degree
0,1.0,Alexandria,Heller,4.0,81.337515,NaN,79.557508,86.677535,NaN
1,2.0,Cassie,Liska,5.0,79.137364,NaN,78.867115,79.542738,NaN
2,3.0,Hayley,Frank,3.0,78.793700,83.972245,76.204427,NaN,NaN
3,4.0,Maxwell,Kotch,3.0,71.481003,70.929801,74.157066,69.356144,NaN
4,5.0,Dylan,Boehm,3.0,76.718650,NaN,100.000000,64.494845,65.661106


# I did it!

### I must say that in retrospect, Python is not that hard. It's even easy once you understand the right flow .